MINIMISE 
$\\f_1(x) = 10x_1 + 2x_2\\$
$f_2(x) = 3x_1 ^ 2 + 2x_2 ^ 2$ 

SUBJECT TO
$\\ x_1 \geq 2\\$
$ x_2 \geq 3 \\$
$ x_1 + x_2 \leq 12$

Both f_1 and f_2 are continously differentiable 


Aim of problem
min f(x) sub to contraints
Quasi-Newton methods
B_n approx = nabla ^2 f(x_n)



In [1]:
#d
import numpy as np
from scipy.optimize import minimize

def search_dir(x, grads, B,n_args, printRes = False):

    def obj(d,B,grads, x):
        return np.max([(grad(x)).T @ d + 0.5 * d.T @ B_j @ d for grad, B_j in zip(grads, B)])
    
    
    result = minimize(obj, d0, args=(B, grads))
    
     # Initial guess for d
    d0 = np.zeros(n_args)

    # Minimize the objective function
    result = minimize(obj, d0, args=(B, grads,x))

    dOpt = result.x
    thetaOpt = result.fun

    return dOpt, thetaOpt



In [2]:
def Delta(grads,x,d):
    return np.max([grad(x).T @ d for grad in grads])

In [3]:
def Armijo(F,grads, x, alpha, d, c1):
    for f in F:
        if f(x + alpha * d) > f(x) + c1 * alpha * Delta(grads, x, d):
            return False
    return True

In [4]:
def Curvature(F, grads, x, alpha, d, c2):
    if Delta(grads,x + alpha * d,d) >= c2 * Delta(grads, x,d):
        return True
    return False

In [5]:
def WLinesearch(F,grads, x,d,c1 = 0.1, c2 = 0.9):
    alpha = 1
    alpha_bar = 0
    alpha_hat = 0
    armijo_test = Armijo(F, grads, x, alpha, d, c1)
    curvature_test = Curvature(F, grads, x, alpha, d, c2)

    while not armijo_test or not curvature_test:
        if not armijo_test:
            alpha_bar = alpha
            alpha = (alpha_bar + alpha_hat) / 2
        elif not curvature_test:
            alpha_bar = alpha

            if alpha_hat == 0:
                alpha = 2 * alpha_bar
            else:
                alpha = (alpha_bar + alpha_hat) / 2
        
        armijo_test = Armijo(F, grads, x, alpha, d, c1)
        curvature_test = Curvature(F, grads, x, alpha, d, c2)

    return alpha

In [6]:
import array


def BFGS(F, grads,B0, x0, x1):
    #Preperation
    y = np.array([grad(x1) - grad(x0) for grad in grads])
    s = x1 - x0
    if s.T @ y > 0:
        p = 1/(s.T @ y)
    else:
        p = 1/ ( Delta(grads,x1,s) - np.array([grad(x0) for grad in grads]).T @ s)


    #Update B
    

    

In [7]:
def BFGS(F,grads, x, B, c1, c2, iter = 100, tol = 1e-6):

    n_args = len(x)
    x0 = x
    B0 = B
    for i in range(iter):

        #Step 1: Search direction
        d,theta = search_dir(x0, grads, B0, n_args)

        #Stopping criterion
        if abs(theta) < tol:
            return x
        
        #Line search
        alpha = WLinesearch(F, grads, x, d, c1, c2)
        x_new = x + alpha * d

        sk = x_new - x

        #B_new = B + (grads(x_new) @ grads(x_new).T) / (grads(x_new).T @ d) - (B @ d @ d.T @ B) / (d.T @ B @ d)